In [17]:
import numpy as np
import pandas as pd

import implicit

from google.cloud import storage

from pandas.io import gbq

import pickle

In [3]:
#client = storage.Client(project='umg-comm-tech-dev')
#bucket = client.get_bucket('umg-comm-tech-dev')

In [4]:
class MacOSFile(object):

    def __init__(self, f):
        self.f = f

    def __getattr__(self, item):
        return getattr(self.f, item)

    def read(self, n):
        # print("reading total_bytes=%s" % n, flush=True)
        if n >= (1 << 31):
            buffer = bytearray(n)
            idx = 0
            while idx < n:
                batch_size = min(n - idx, 1 << 31 - 1)
                # print("reading bytes [%s,%s)..." % (idx, idx + batch_size), end="", flush=True)
                buffer[idx:idx + batch_size] = self.f.read(batch_size)
                # print("done.", flush=True)
                idx += batch_size
            return buffer
        return self.f.read(n)

    def write(self, buffer):
        n = len(buffer)
        print("writing total_bytes=%s..." % n, flush=True)
        idx = 0
        while idx < n:
            batch_size = min(n - idx, 1 << 31 - 1)
            print("writing bytes [%s, %s)... " % (idx, idx + batch_size), end="", flush=True)
            self.f.write(buffer[idx:idx + batch_size])
            print("done.", flush=True)
            idx += batch_size


def pickle_dump(obj, file_path):
    with open(file_path, "wb") as f:
        return pickle.dump(obj, MacOSFile(f), protocol=pickle.HIGHEST_PROTOCOL)


def pickle_load(file_path):
    with open(file_path, "rb") as f:
        return pickle.load(MacOSFile(f))

In [8]:
model = pickle_load('/Users/meshchd/Downloads/saved_model.pkl')

In [9]:
df = pd.read_csv('/Users/meshchd/Downloads/500UMeISRCs.csv')

In [10]:
df.head()

,isrc,artist,title
0,GBAAA8500070,Simple Minds,Alive And Kicking
1,USUG11401961,Madonna,Bitch I'm Madonna
2,GBUM71028665,James Blake,Limit To Your Love
3,USCH38400069,Pat Benatar,Shadows Of The Night
4,USPR36905204,The Velvet Underground,Pale Blue Eyes


In [11]:
df.to_gbq('amplify.ume500_isrcs', project_id='umg-comm-tech-dev', if_exists='replace')

1it [00:05,  5.35s/it]


In [12]:
get_existing_tags = """
SELECT
  isrc,
  confidence,
  path,
  source,
  taxonomy_node_id,
  value
FROM
  `umg-metadata.ircam.amplify_tem_v3_3_v`
WHERE
  isrc IN (
  SELECT
    isrc
  FROM
    `umg-comm-tech-dev.amplify.ume500_isrcs`
  GROUP BY
    isrc)
"""

In [13]:
tags = gbq.read_gbq(get_existing_tags, project_id='umg-comm-tech-dev', dialect='standard')

In [14]:
tags.head()

,isrc,confidence,path,source,taxonomy_node_id,value
0,GRE018000028,1.000000,Anatomy/Harmony/Root,Auto,acf5ddcf-0890-4647-8823-16522e66b541,A
1,GRE018000028,0.544659,Energy/Emotion,Auto,3adcefb7-b1c1-4f4d-a40c-4032c8d35aca,Negative
2,GRE018000028,0.544659,Energy/Emotion/Negative,Auto,77822969-740e-4f56-9ea0-09de404dc071,Sad
3,GRE018000028,1.000000,Energy/Intensity/Medium,Manual,0b5909f2-c61f-43f2-a51d-a473fb01a768,Medium Intensity
4,GRE018000028,0.631046,Performance/Ensemble/Ensemble Configuration,Auto,6df35aeb-4c6f-448e-8d14-1967aa24a098,Voice & Accompaniment


In [15]:
tags.describe(include='all')

,isrc,confidence,path,source,taxonomy_node_id,value
count,30518,30518.000000,30518,30518,30518,30518
unique,491,NaN,100,2,256,733
top,GBF075820000,NaN,Performance/Ensemble/Ensemble Section,Auto,ca8d3fea-e3cc-4c38-abd3-0a32e49a311f,Percussion
freq,274,NaN,4180,29948,1062,1775
mean,NaN,0.852100,NaN,NaN,NaN,NaN
std,NaN,0.180539,NaN,NaN,NaN,NaN
min,NaN,0.301012,NaN,NaN,NaN,NaN
25%,NaN,0.732352,NaN,NaN,NaN,NaN
50%,NaN,0.939958,NaN,NaN,NaN,NaN
75%,NaN,1.000000,NaN,NaN,NaN,NaN


In [16]:
tags.groupby(by='isrc').value.count().describe()

count    491.000000
mean      62.154786
std       26.924090
min        6.000000
25%       50.000000
50%       58.000000
75%       84.000000
max      274.000000
Name: value, dtype: float64

In [30]:
tags_in_recommender = [isrc if isrc in model.isrcs else '' for isrc in tags.isrc.unique()]

In [31]:
tags_in_recommender = list(set(tags_in_recommender))

In [32]:
len(tags_in_recommender)

168

In [71]:
tags_in_recommender[1]

'GBAYE8200083'

In [72]:
def get_similar_items(seed_isrc, n=100):
    
    item_id = model.isrcs.index(seed_isrc)

    # Get the item row for the isrc
    item_vec = model.isrc_vecs[item_id].T

    # Calculate the similarity score
    # and select the top 100 most similar.
    scores = model.isrc_vecs.dot(item_vec).reshape(1,-1)[0]
    top_100 = np.argsort(scores)[::-1][:n]

In [87]:
similar_isrcs = []
artists = []
names = []
isrc_scores = []

# Get and print the actual artists names and scores
for idx in list(top_100):

    isrc = model.isrcs[idx]
    lookup = model.isrc_lookup[model.isrc_lookup.isrc==isrc].iloc[0]
    
    similar_isrcs.append(isrc)
    artists.append(lookup[1])
    names.append(lookup[2])
    isrc_scores.append(scores[idx])

In [89]:
similar = pd.DataFrame({'isrc': similar_isrcs, 'artist': artists, 'track_name': names, 'score': isrc_scores})
similar['seed_isrc']='GBAYE8200083'

In [90]:
similar

,isrc,artist,track_name,score,seed_isrc
0,GBAYE8200083,Thomas Dolby,She Blinded Me With Science,0.012011,GBAYE8200083
1,CHD771800006,Alejandro Reyes,Solamente,0.010726,GBAYE8200083
2,ROARA1700019,Andra,Shukar,0.010030,GBAYE8200083
3,USSD11900123,Descemer Bueno,Nos Fuimos Lejos (feat. Ece Seçkin & El Micha)...,0.009937,GBAYE8200083
4,ushm20708304,Emile Pandolfi,Once Upon a December (from Anastasia),0.009774,GBAYE8200083
5,USMC18416586,Harold Faltermeyer,"Axel F - From ""Beverly Hills Cop""",0.009662,GBAYE8200083
6,USOEG1883101,Smash Mouth,Unity,0.009605,GBAYE8200083
7,USMO10000307,Rockwell,Somebody's Watching Me,0.009524,GBAYE8200083
8,GBAYE8200051,Duran Duran,Hungry Like The Wolf,0.009520,GBAYE8200083
9,USA5W1800322,Kirk Franklin,Love Theory,0.009465,GBAYE8200083
